In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')
# Find more styles here
# plt.style.available

In [2]:
train = pd.read_csv("train.csv", header = 0 , index_col=0)


In [3]:
#Adding new variables
train["Gender"] = train["Sex"].map({"female" : 0, "male" : 1})
train["Got on"] = train["Embarked"].map({"C" : 0, "Q" : 1, "S" : 2})
train["Family_size"] = train["Parch"] + train["SibSp"]

#add age to NaN values based on Pclass and Gender
X_age = train[["Pclass", "Gender"]][train["Age"] > 0]
Y_age = train["Age"][train["Age"] > 0]
X_age.head()
from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg = reg.fit(X_age, Y_age)
new_age = train[["Pclass", "Gender"]]
train["Pred_age"] = reg.predict(new_age)

train["Age"] = train["Age"].fillna(train["Pred_age"])

X_fare = train[["Pclass", "Gender", "Age"]][train["Fare"] > 0]
Y_fare = train["Fare"][train["Fare"] > 0]

from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg = reg.fit(X_fare, Y_fare)
new_fare = train[["Pclass", "Gender", "Age"]]
train["Pred_fare"] = reg.predict(new_fare)

train["Fare"] = train["Fare"].fillna(train["Pred_fare"])


#fills Got on NaN to 0
train["Got on"] = train["Got on"].fillna(0)

train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 16 columns):
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            891 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
Gender         891 non-null int64
Got on         891 non-null float64
Family_size    891 non-null int64
Pred_age       891 non-null float64
Pred_fare      891 non-null float64
dtypes: float64(5), int64(6), object(5)
memory usage: 118.3+ KB


In [ ]:
%matplotlib inline
import seaborn as sns
sns.set(style="whitegrid", palette="pastel", color_codes=True)

# Draw a nested violinplot and split the violins for easier comparison
sns.violinplot(x="Pclass", y="Age", hue="Sex", data=train, split=True,
               inner="quart", palette={"male": "b", "female": "y"})
sns.despine(left=True)


In [ ]:
sns.violinplot(x="Pclass", y="Fare", hue="Survived", data=train, split=True,
               inner="quart", palette={1: "g", 0: "r"})
sns.despine(left=True)

In [ ]:
sns.factorplot(x="Pclass", y="Survived", hue="Sex", data=train,
                   size=6, kind="bar", palette="muted")
sns.despine(left = True)

In [ ]:
sns.factorplot(x="Family_size", y="Survived", data=train,
                   size=6, kind="bar", palette="muted")

In [ ]:
test = pd.read_csv("test.csv", header = 0 , index_col=0)
#Adding new variables
test["Gender"] = test["Sex"].map({"female" : 0, "male" : 1})
test["Got on"] = test["Embarked"].map({"C" : 0, "Q" : 1, "S" : 2})
test["Family_size"] = test["Parch"] + test["SibSp"]

#add age to NaN values based on Pclass and Gender
X_age = test[["Pclass", "Gender"]][test["Age"] > 0]
Y_age = test["Age"][test["Age"] > 0]
X_age.head()
from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg = reg.fit(X_age, Y_age)
new_age = test[["Pclass", "Gender"]]
test["Pred_age"] = reg.predict(new_age)

test["Age"] = test["Age"].fillna(test["Pred_age"])

X_fare = test[["Pclass", "Gender", "Age"]][test["Fare"] > 0]
Y_fare = test["Fare"][test["Fare"] > 0]
X_fare.head()
from sklearn import linear_model
reg = linear_model.Lasso(alpha = 0.1)
reg = reg.fit(X_fare, Y_fare)
new_fare = test[["Pclass", "Gender", "Age"]]
test["Pred_fare"] = reg.predict(new_fare)

test["Fare"] = test["Fare"].fillna(test["Pred_fare"])

#fills Got on NaN to 0
test["Got on"] = test["Got on"].fillna(0)
test.info()

In [ ]:
train[Target].head()

In [4]:

#Sets what features to use in the estimators
Features = train[["Pclass", "Gender", "Age","Fare", "Family_size", "Got on"]]
#Features = train[["Pclass", "Gender", "Age"]]
Target = train["Survived"]



from sklearn import svm
from sklearn.model_selection import GridSearchCV

#Instanciates an estimator
clf = svm.SVC()

#Makes the param grid
C = [1, 2, 4, 8]
kernel = ["rbf"]
gamma = [0.1, 0.01, 0.001]
param_grid = dict(C = C, kernel = kernel, gamma = gamma)

#instantiate the crossvalidation search
grid = GridSearchCV(clf, param_grid= param_grid, cv = 8, scoring = "accuracy", n_jobs=-1)

#Runs the crossvalitation on the data
grid.fit(Features, Target)

#Prints relevent information of the best estimator found
print(grid.best_score_)
print(grid.best_params_)

0.751964085297
{'C': 8, 'kernel': 'rbf', 'gamma': 0.01}


In [ ]:

#Runs the prediction
X_survived = train[["Pclass", "Gender", "Age", "Fare", "Family_size", "Got on"]]
Y_survived = train["Survived"]

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators= 10)
clf = clf.fit(X_survived, Y_survived)

test_features = test[["Pclass", "Gender", "Age", "Fare", "Family_size", "Got on"]]

test["Survived"] = clf.predict(test_features)
test.head()

#Saves to csv that works for Kaggle
prediction = test["Survived"]
prediction.to_csv( "prediction.csv", index_label="PassengerId", index = True, header = True)


In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
clf = svm.SVC()

C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)

grid = RandomizedSearchCV(clf, param_distributions=param_grid, cv = 5, n_iter=10, scoring = "accuracy", n_jobs = -1)

grid.fit(Features, Target)

print(grid.best_score_)
print(grid.best_params_)